In [9]:
pip install -r requirements.txt

In [6]:
!pip install bertviz

     |████████████████████████████████| 163kB 18.5MB/s 
     |████████████████████████████████| 1.2MB 47.8MB/s 
     |████████████████████████████████| 133kB 60.9MB/s 
     |████████████████████████████████| 81kB 10.5MB/s 
     |████████████████████████████████| 7.4MB 56.1MB/s 
ERROR: botocore 1.20.53 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [2]:
import os
import time

# Data related
import numpy as np
import pandas as pd
from datasets import load_dataset
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics.pairwise import cosine_distances

from transformers.trainer_utils import set_seed
from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import TFBertForSequenceClassification
from transformers import TFBertForMaskedLM
from transformers.data.data_collator import DataCollatorForLanguageModeling
from transformers.modeling_tf_utils import TFMaskedLanguageModelingLoss

from bertviz import head_view
from collections import Counter

# TF 2.4
# Cuda 11.0
# CuDNN 8.0.5 (CUDA 11.0)
# at nvidia-smi if you encounter Failed to initialize NVML: Driver/library version mismatch
# simply restart the computer and see if it goes away
import tensorflow as tf
import tensorflow.keras.backend as K
import torch
from tensorflow.keras.losses import SparseCategoricalCrossentropy


print(tf.config.list_physical_devices('GPU'))

random_seed=4321

set_seed(random_seed)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [149]:
dataset = load_dataset('csv', data_files='240min_Amazon_news_direction_updated.csv')

Using custom data configuration default-cc3504bc683bd43a


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-cc3504bc683bd43a/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [150]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['news', 'direction'],
        num_rows: 2032
    })
})


In [151]:
for s,l in zip(dataset['train']['news'][:10], dataset['train']['direction'][:10]):
    print(s)
    print('\tLabel: {}'.format(l))

andr januari nasdaq surg big name tech stock continu show dominance. well fang stock perform could room run scorecard facebook fb amazon amzn netflix nflx googl parent alphabet googl appl aapl . facebook x facebook gain . stock last broke base octob seen somewhat sluggish sideway action sinc then. facebook share remain within buy rang . entri lost support key day line. facebook also new flat base . buy point. investor may want focus entri now. amazon amazon share increas stock could soon present fresh buy opportunity. friday close stock complet five week base build minimum amount time take base form. new buy point . . late stage base mean higher odd failing. ibd take look high qualiti stock potenti big gain make sure check leaderboard curat list top growth name near potenti buy points. new ibd take free trial leaderboard. netflix netflix rose year built sever base along run. share close friday back day line tri build right side base . entry. alphabet googl parent alphabet climb least f

In [154]:
print(pd.Series(dataset['train']['direction']).value_counts())

1    1231
0     801
dtype: int64


In [155]:
inputs, labels = np.array(dataset['train']['news']).reshape(-1,1), np.array(dataset['train']['direction'])

In [165]:
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

n=100 # Number of instances for each class for test/validation sets

# Define a random undersample that will give equal amounts of data for each label randomly
rus = RandomUnderSampler(sampling_strategy={0:n, 1:n}, random_state=random_seed)
rus.fit_resample(inputs, labels)

# Get test indices from the random undersampler
test_inds = rus.sample_indices_
test_x, test_y = inputs[test_inds], np.array(labels)[test_inds]
print("Test statistics")
print(pd.Series(test_y).value_counts())

# Get rest (train + valid)
rest_inds = [i for i in range(inputs.shape[0]) if i not in test_inds]
rest_x, rest_y = inputs[rest_inds], labels[rest_inds]

# Get valid indices from the random undersampler
rus.fit_resample(rest_x, rest_y)
valid_inds = rus.sample_indices_
valid_x, valid_y = rest_x[valid_inds], rest_y[valid_inds]
print("Valid statistics")
print(pd.Series(valid_y).value_counts())

# Rest goes in training
train_inds = [i for i in range(rest_x.shape[0]) if i not in valid_inds]
train_x, train_y = rest_x[train_inds], rest_y[train_inds]
print("Train statistics")
print(pd.Series(train_y).value_counts())

Test statistics
1    100
0    100
dtype: int64
Valid statistics
1    100
0    100
dtype: int64
Train statistics
1    1031
0     601
dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [166]:
test_x[:10,0].tolist()

['las vega music legend lionel richi unlik pitchman newfangl medic devic sprawl consum electron show las vega week. eclect field health care technolog morph mani shape form commerci ventur explor possibl artifici intellig robot vast amount data. getti imag smartphon app small devic life size robot ces saw fair share technolog solutions. tech reshap larg swath multibillion dollar industri like manifest wearabl robot sensor ai conclud panel topic call invis doctor. anoth wild card amazon.com ticker amzn anticip entri pharmaci market. growth project health care tech market plenti make hard put definit figur expert agre enorm market. recent avail inform market research euromonitor intern peg global sale consum health product billion . everybodi come health care slosh money probabl trillion dollar say girish venkatachaliah former ibm watson health execut vice presid machin learn athenahealth athn . appl aapl alphabet googl googl amazon ibm ibm circl industry. amplifi come months. rub modifi

In [167]:
model_bert = {}
tokenizer_bert = {}

In [168]:

from transformers import BertTokenizer, TFBertModel

# Download FinBERT tokenizer
# Before running the model below download the vocab file by following the instruction above
tokenizer_bert['finbert'] = BertTokenizer.from_pretrained(os.path.join('data', 'fin_vocab'))

In [169]:
text = "401(k) Asset Turnover Ratio Blockchain Bankruptcy Demonetization FAANG Stocks Fintech Home Equity Loan Indemnity Insurance Liquidity Macroeconomics"


# Output of FinBERT
encoded_ids = tokenizer_bert['finbert'](text, return_tensors='tf')
encoded_tokens = tokenizer_bert['finbert'].convert_ids_to_tokens(encoded_ids['input_ids'].numpy()[0])
print('FinBERT says ...\n')
print(encoded_tokens)

FinBERT says ...

['[CLS]', '401', '(', 'k', ')', 'asset', 'turnover', 'ratio', 'block', '##chain', 'bankruptcy', 'demo', '##net', '##ization', 'faa', '##ng', 'stocks', 'fin', '##tech', 'home', 'equity', 'loan', 'indemnity', 'insurance', 'liquidity', 'macroeconomic', '##s', '[SEP]']


In [170]:
from transformers import TFBertForSequenceClassification

# Let's now create BERT + classifier model to solve the usecase
num_labels = 2
model_bert['finbert+classifier'] = TFBertForSequenceClassification.from_pretrained(os.path.join('models','finbert'), num_labels=num_labels, from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [171]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_bert['finbert+classifier'].compile(optimizer=optimizer, loss=loss, metrics=tf.keras.metrics.SparseCategoricalAccuracy())
model_bert['finbert+classifier'].summary()

Model: "tf_bert_for_sequence_classification_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109751808 
_________________________________________________________________
dropout_455 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,753,346
Trainable params: 109,753,346
Non-trainable params: 0
_________________________________________________________________


In [172]:
seq_length = []

remove_list_train = []
for indx, val in np.ndenumerate(train_x):
  if val == None:
    print("yooooooooo")
    print(val)
    remove_list_train.append(indx)

    
new_train_x = np.delete(train_x, remove_list_train)
new_train_y = np.delete(train_y, remove_list_train)

remove_list_valid = []
for indx, val in np.ndenumerate(valid_x):
  if val == None:
    print("yooooooooo")
    print(val)
    remove_list_valid.append(indx)

new_valid_x = np.delete(valid_x, remove_list_valid)
new_valid_y = np.delete(valid_y, remove_list_valid)


for x in new_train_x:
    # print(type(x))
    # print(x)
    seq_length.append(len(tokenizer_bert['finbert'](x[0])['input_ids']))

train_x = new_train_x
print(len(train_x))
print(len(train_y))
train_y = new_train_y
print(len(train_y))

valid_x = new_valid_x
valid_y = new_valid_y

print(pd.Series(seq_length).describe(percentiles=[0.25,0.5,0.75,0.9]))
print(pd.Series(seq_length).median())

1632
1632
1632
count    1632.0
mean        3.0
std         0.0
min         3.0
25%         3.0
50%         3.0
75%         3.0
90%         3.0
max         3.0
dtype: float64
3.0


In [173]:
def examples_to_features(x, tokenizer):
    """ Convert an example to a dictionary BERT model accepts """
    res = tokenizer(x, padding=True, truncation=True, max_length=50, return_tensors='tf')
    return dict(list(res.items()))


a = train_x.reshape(-1).tolist()
print(type(a))
print(type(a[0]))
print(a[0])
train_tokens = examples_to_features(train_x.reshape(-1).tolist(), tokenizer_bert['finbert'])
train_labels = train_y.reshape(-1,1)

valid_tokens = examples_to_features(valid_x.reshape(-1).tolist(), tokenizer_bert['finbert'])
valid_labels = valid_y.reshape(-1,1)


# Sample data
sample_text = "Company shares are down"
# sample_text = 
print(examples_to_features(sample_text, tokenizer_bert["finbert"]))

<class 'list'>
<class 'str'>
andr januari nasdaq surg big name tech stock continu show dominance. well fang stock perform could room run scorecard facebook fb amazon amzn netflix nflx googl parent alphabet googl appl aapl . facebook x facebook gain . stock last broke base octob seen somewhat sluggish sideway action sinc then. facebook share remain within buy rang . entri lost support key day line. facebook also new flat base . buy point. investor may want focus entri now. amazon amazon share increas stock could soon present fresh buy opportunity. friday close stock complet five week base build minimum amount time take base form. new buy point . . late stage base mean higher odd failing. ibd take look high qualiti stock potenti big gain make sure check leaderboard curat list top growth name near potenti buy points. new ibd take free trial leaderboard. netflix netflix rose year built sever base along run. share close friday back day line tri build right side base . entry. alphabet googl 

In [174]:
def get_class_weights(labels):
    """ Get class weights for classes. Less data more weight """
    n_freq = pd.Series(labels.reshape(-1)).value_counts()
    dct = {}
    for lbl in [0,1]:
        if lbl==0:
            others = [1]
        elif lbl==1:
            others = [0]
        dct[lbl] = n_freq[others].sum()/n_freq.sum()
    return dct

print(get_class_weights(train_labels))

{0: 0.6317401960784313, 1: 0.36825980392156865}


In [175]:
# Finetune FinBert on the financial sentiment classification task
import time

t1 = time.time()

model_bert['finbert+classifier'].fit(
    train_tokens, train_labels, 
    validation_data=(valid_tokens, valid_labels), 
    class_weight=get_class_weights(train_labels),
    epochs=10
)

t2 = time.time()
print("It took {} seconds to complete the training".format(t2-t1))

Epoch 1/10
51/51 [==============================] - 30s 388ms/step - loss: 0.3233 - sparse_categorical_accuracy: 0.5413 - val_loss: 0.7155 - val_sparse_categorical_accuracy: 0.5250
Epoch 2/10
51/51 [==============================] - 17s 331ms/step - loss: 0.2893 - sparse_categorical_accuracy: 0.6223 - val_loss: 0.6629 - val_sparse_categorical_accuracy: 0.6550
Epoch 3/10
51/51 [==============================] - 17s 326ms/step - loss: 0.1813 - sparse_categorical_accuracy: 0.8231 - val_loss: 0.7049 - val_sparse_categorical_accuracy: 0.6750
Epoch 4/10
51/51 [==============================] - 16s 323ms/step - loss: 0.0791 - sparse_categorical_accuracy: 0.9427 - val_loss: 0.7515 - val_sparse_categorical_accuracy: 0.7000
Epoch 5/10
51/51 [==============================] - 16s 322ms/step - loss: 0.0684 - sparse_categorical_accuracy: 0.9577 - val_loss: 0.9220 - val_sparse_categorical_accuracy: 0.7050
Epoch 6/10
51/51 [==============================] - 17s 325ms/step - loss: 0.0367 - sparse_cate

In [176]:
if not os.path.exists(os.path.join('models', 'finbert_classifier')):
    os.makedirs(os.path.join('models', 'finbert_classifier'))
if not os.path.exists(os.path.join('tokenizers', 'finbert_classifier')):
    os.makedirs(os.path.join('tokenizers', 'finbert_classifier'))

# Save the modle
model_bert['finbert+classifier'].save_pretrained(os.path.join('models', 'finbert_classifier'))

# Save the tokenizer
tokenizer_bert['finbert'].save_pretrained(os.path.join('tokenizers', 'finbert_classifier'))

('tokenizers/finbert_classifier/tokenizer_config.json',
 'tokenizers/finbert_classifier/special_tokens_map.json',
 'tokenizers/finbert_classifier/vocab.txt',
 'tokenizers/finbert_classifier/added_tokens.json')

In [177]:
# Load the model
model_bert['finbert+classifier'] = TFBertForSequenceClassification.from_pretrained(
    os.path.join('models','finbert_classifier'), num_labels=num_labels)

# Load the tokenizer
tokenizer_bert['finbert'] = BertTokenizer.from_pretrained(os.path.join('tokenizers', 'finbert_classifier'))

# Compile the model so it can be used for predictions
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_bert['finbert+classifier'].compile(optimizer=optimizer, loss=loss, metrics=tf.keras.metrics.SparseCategoricalAccuracy())

Some layers from the model checkpoint at models/finbert_classifier were not used when initializing TFBertForSequenceClassification: ['dropout_455']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at models/finbert_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [178]:

a = test_x.reshape(-1).tolist()
b = test_y.reshape(-1,1)
print(type(a))
b_list = []
if type(a[0]) == str:
  print("HIIIII")
for indx, val in enumerate(a):
  if type(val) != str:
    print("yooooooooo")
    a.pop(indx)
    b_list.append(indx)
    
new_b = np.delete(b, b_list)

    
# print(a[0])
test_tokens = examples_to_features(a, tokenizer_bert['finbert'])
test_labels = new_b

model_bert['finbert+classifier'].evaluate(test_tokens, test_labels)

<class 'list'>
HIIIII
7/7 [==============================] - 3s 101ms/step - loss: 1.6014 - sparse_categorical_accuracy: 0.6950


[1.3053195476531982, 0.7300000190734863]